In [119]:
import warnings
warnings.filterwarnings('ignore')

In [120]:
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
import os
from PIL import Image
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import random
import time
import timm

from sklearn.model_selection import train_test_split

In [121]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

seed_everything()

In [122]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list = None, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        img_path = './open'+img_path[1:]
        image = Image.open(img_path).convert('RGB')
        
        if self.transforms is not None:
            image = self.transforms(image)
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [123]:
def get_train_val(df):
    df = df.sample(frac=1)
    train_len = int(len(df) * 0.8)
    train_df = df[:train_len]
    val_df = df[train_len:]
    
    train_x, train_y  = train_df['img_path'].to_list(), train_df.iloc[:,2:].values
    val_x, val_y = val_df['img_path'].to_list(), val_df.iloc[:,2:].values
 
    return train_x,train_y, val_x,val_y


def get_dataloader(df,mode,batch_size):
    """
    데이터프레임을 dataloader형태로 반환하는 함수
    """
    
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    
    # augmentations 적용 
    train_augmentations = transforms.Compose([   
#                     transforms.Resize((640, 640)),
                    #transforms.RandomResizedCrop(224),
#                     transforms.RandomCrop(224),
#                     transforms.RandomRotation(30),
                    transforms.RandomGrayscale(p=0.4),
#                     transforms.Grayscale(num_output_channels=3),
                    transforms.RandomAffine(45, shear=0.2),
                    transforms.ColorJitter(),
                    transforms.RandomHorizontalFlip(),
                    #transforms.Lambda(utils.randomColor),
                    #transforms.Lambda(utils.randomBlur),
                    #transforms.Lambda(utils.randomGaussian),
                    transforms.ToTensor(),
                    normalize,
            ])
    valid_augmentations = transforms.Compose([
                    transforms.Resize((224, 224)),
                    #transforms.CenterCrop(299),
                    #transforms.Grayscale(num_output_channels=3),
                    transforms.ToTensor(),
                    normalize
        ])
    
    if mode == 'TRAIN':

        train_x,train_y, val_x,val_y = get_train_val(df)
        # Data Loader
        train_dataset = CustomDataset(train_x,train_y, transforms = train_augmentations)
        valid_dataset = CustomDataset(val_x, val_y, transforms = valid_augmentations)
        
        train_data_loader = DataLoader(
            train_dataset,
            batch_size = batch_size,
            shuffle = True,
            num_workers = 16,
            pin_memory = True
        )
        valid_data_loader = DataLoader(
            valid_dataset,
            batch_size = batch_size,
            shuffle = False,
            num_workers = 16,
            pin_memory = True
        )
        return train_data_loader, valid_data_loader
    
    else :
        test_dataset = CustomDataset(img_path_list = df['img_path'], transforms = valid_augmentations)
        test_data_loader = DataLoader(
            test_dataset,
            batch_size = batch_size,
            shuffle = False,
            num_workers = 16,
            drop_last = False
        )
        return test_data_loader
        

    

In [214]:
class BaseModel(nn.Module):
    def __init__(self, model_type, num_classes=10):
        super(BaseModel, self).__init__()
        if model_type == 'm' :
            self.backbone = models.efficientnet_v2_m(pretrained=True)
        if model_type == 'l' :
            self.backbone = models.efficientnet_v2_l(pretrained=True)  
    
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = F.sigmoid(self.classifier(x))
        return x

In [215]:
test_df = pd.read_csv('open/test.csv')
test_loader = get_dataloader(test_df,mode='TEST',batch_size = 64)

In [216]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            
            preds = probs > 0.5
            preds = preds.astype(int)
            predictions += preds.tolist()
    return predictions

# soft voting

In [217]:
predictions = []

model_list_m = ['checkpoint/effiv7_175_8.392196619481787.pth','checkpoint/effiv2_Adam_1-fold_191_8.52181449063465.pth','checkpoint/effiv7_178_8.499276629949021.pth']
model = BaseModel(model_type = 'm')

for model_weight in model_list:
    model.load_state_dict(torch.load(model_weight))
    device = torch.device('cuda:3')
    model.to(device)
    model.eval()

    prediction = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)

            probs = model(imgs)
            probs  = probs.cpu().detach().numpy()
    #         preds = probs > 0.5
    #         preds = preds.astype(int)
            prediction += probs.tolist()
        predictions.append(prediction)
        

100%|██████████| 23/23 [00:07<00:00,  3.23it/s]


In [219]:
model_list_l = ['checkpoint/effiv2L_1-fold_154_8.492884039012917.pth']
model = BaseModel(model_type = 'l')
for model_weight in model_list:
    model.load_state_dict(torch.load(model_weight))
    device = torch.device('cuda:3')
    model.to(device)
    model.eval()

    prediction = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)

            probs = model(imgs)
            probs  = probs.cpu().detach().numpy()
    #         preds = probs > 0.5
    #         preds = preds.astype(int)
            prediction += probs.tolist()
        predictions.append(prediction)
        

100%|██████████| 23/23 [00:11<00:00,  1.95it/s]


In [228]:
pred = []
for idx in range(len(predictions[0])):
    temp = None
    for idx_2 in range(len(predictions)):
        if temp is not None :
            temp += np.array(predictions[idx_2][idx])
        else : 
            temp = np.array(predictions[idx_2][idx])
    
    temp = temp / (len(model_list_m) + len(model_list_l))
    preds = temp > 0.5
    preds = preds.astype(int)
    pred.append(preds.tolist())

In [229]:
submit = pd.read_csv('./open/sample_submission.csv')
submit.iloc[:,1:] = pred
submit.head()


,id,A,B,C,D,E,F,G,H,I,J
0,TEST_00000,0,1,0,0,0,0,1,0,1,0
1,TEST_00001,0,1,0,0,0,1,0,0,0,0
2,TEST_00002,1,1,0,0,1,1,1,1,0,1
3,TEST_00003,1,1,1,0,0,1,0,1,1,0
4,TEST_00004,0,0,0,0,1,0,0,0,0,0


In [230]:
submit.to_csv('./soft_voting_test.csv', index=False)

# hardvoting

In [192]:

model = BaseModel()
model_list_m = ['checkpoint/effiv7_175_8.392196619481787.pth','checkpoint/effiv2_Adam_1-fold_191_8.52181449063465.pth','checkpoint/effiv7_178_8.499276629949021.pth']
predictions = []
for model_weight in model_list_m:
    model.load_state_dict(torch.load(model_weight))
    device = torch.device('cuda:2')
    model.to(device)
    model.eval()

    prediction = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)

            probs = model(imgs)
            probs  = probs.cpu().detach().numpy()
            preds = probs > 0.5
            preds = preds.astype(int)
            prediction += preds.tolist()
        predictions.append(prediction)
        

100%|██████████| 23/23 [00:05<00:00,  4.59it/s]


In [ ]:
model_list_l = ['checkpoint/effiv2L_1-fold_154_8.492884039012917.pth']
model = BaseModel(model_type = 'l')
for model_weight in model_list:
    model.load_state_dict(torch.load(model_list_l))
    device = torch.device('cuda:3')
    model.to(device)
    model.eval()

    prediction = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)

            probs = model(imgs)
            probs  = probs.cpu().detach().numpy()
    #         preds = probs > 0.5
    #         preds = preds.astype(int)
            prediction += probs.tolist()
        predictions.append(prediction)
        

In [204]:
pred = []
for idx in range(len(predictions[0])):
    temp = (np.array(predictions[0][idx]) + np.array(predictions[1][idx]) + np.array(predictions[2][idx]))/(len(model_list_m)+len(model_list_l))
    preds = temp > 0.5


    preds = preds.astype(int)
    pred.append(preds.tolist())

In [206]:
submit = pd.read_csv('./open/sample_submission.csv')
submit.iloc[:,1:] = pred
submit.head()


,id,A,B,C,D,E,F,G,H,I,J
0,TEST_00000,0,1,0,0,0,0,1,0,1,0
1,TEST_00001,0,1,0,0,0,1,0,0,0,0
2,TEST_00002,1,1,0,0,1,1,1,1,0,1
3,TEST_00003,1,1,1,0,0,1,0,1,1,0
4,TEST_00004,0,0,0,0,1,0,0,0,0,0


In [207]:
submit.to_csv('./hard_voting_test.csv', index=False)

---